<a href="https://colab.research.google.com/github/coda-nsit/BERT_experiments/blob/master/BioBERT_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References
1. ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory: https://github.com/tensorflow/tensorflow/issues/15604
2. Tensorflow model not found error while converting it to PyTorch model: https://github.com/tensorflow/models/issues/2676#issuecomment-421765031
3. Install cuda-9-0: https://developer.nvidia.com/cuda-90-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1604&target_type=debnetwork
4. Tensorflow to PyTorch: https://huggingface.co/transformers/converting_tensorflow_models.html

# Install and download libraries

In [1]:
!pip3 install transformers
!git clone https://github.com/dmis-lab/biobert.git
!pip3 install -r biobert/requirements.txt

     |████████████████████████████████| 552kB 24.6MB/s 
     |████████████████████████████████| 3.7MB 48.5MB/s 
     |████████████████████████████████| 870kB 56.9MB/s 
     |████████████████████████████████| 1.0MB 64.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=d9d46c2d6795a9e01cbec306166f8f6ca7c7965c8446c50218c64fa4088a5de1
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
Cloning into 'biobert'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 293 (delta 7), reused 0 (delta 0), pack-reused 278
Receiving objects: 100% (293/293), 489.78 KiB | 554.00 KiB/s, done.
Resolving deltas: 100% (169/169), done.
     |████████████████████████████████| 258.8MB 60kB/s 
     |████████████████████████████████| 11.7MB 64.2MB/s 
     |██████████████████████████

BioBERT requires Tensorflow==1.11, which uses Cuda 9.0, so uninstall the latest version and reinstall the old version

In [1]:
!apt remove cuda
!apt-get purge nvidia-cuda*
!apt autoremove
!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!apt-get update
!apt install cuda-9-0
!rm cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'cuda' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-cuda-toolkit' for glob 'nvidia-cuda*'
Note, selecting 'nvidia-cuda-dev' for glob 'nvidia-cuda*'
Note, selecting 'nvidia-cuda-doc' for glob 'nvidia-cuda*'
Note, selecting 'nvidia-cuda-gdb' for glob 'nvidia-cuda*'
Package 'nvidia-cuda-dev' is not installed, so not removed
Package 'nvidia-cuda-doc' is not installed, so not removed
Package 'nvidia-cuda-gdb' is not installed, so not removed
Package 'nvidia-cuda-toolkit' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 25 not upgrad

### Load the BioBERT weights 
The weights are saved in google drive. Not using `wget` cause, the weights are saved in google drive, downloading it is difficult.

In [64]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!cp /gdrive/"My Drive"/OncampusJob/biobert_v1.1_pubmed.tar.gz .
!tar -xzf biobert_v1.1_pubmed.tar.gz
!rm biobert_v1.1_pubmed.tar.gz
%mv  biobert_v1.1_pubmed biobert

In [68]:
!ls

bert_base  biobert  sample_data


In [69]:
%env BIO_BERT_BASE_DIR=/content/biobert
!transformers-cli convert \
  --model_type bert \
  --tf_checkpoint $BIO_BERT_BASE_DIR/model.ckpt-1000000 \
  --config $BIO_BERT_BASE_DIR/bert_config.json \
  --pytorch_dump_output $BIO_BERT_BASE_DIR/pytorch_model.bin

env: BIO_BERT_BASE_DIR=/content/biobert
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:5

### Load the Tensorflow BERT weights

In [25]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
!unzip cased_L-12_H-768_A-12.zip
!rm cased_L-12_H-768_A-12.zip
!mv cased_L-12_H-768_A-12 bert_base

--2020-04-02 01:58:19--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 385.53M  86.6MB/s    in 4.5s    

2020-04-02 01:58:24 (86.6 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [404261442/404261442]

Archive:  cased_L-12_H-768_A-12.zip
   creating: cased_L-12_H-768_A-12/
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: cased_L-12_H-768_A-12/vocab.txt  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: cased_L-12_H-768_A-12/bert_config.json  


In [70]:
!ls bert_base/

bert_config.json		     bert_model.ckpt.index  pytorch_model.bin
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta   vocab.txt


In [28]:
%env BERT_BASE_DIR=/content/bert_base
!transformers-cli convert \
  --model_type bert \
  --tf_checkpoint $BERT_BASE_DIR/bert_model.ckpt \
  --config $BERT_BASE_DIR/bert_config.json \
  --pytorch_dump_output $BERT_BASE_DIR/pytorch_model.bin

env: BERT_BASE_DIR=/content/bert_base
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526

# Get the embeddings

In [71]:
!ls 

bert_base  biobert  sample_data


In [72]:
!ls biobert/

bert_config.json			model.ckpt-1000000.meta
model.ckpt-1000000.data-00000-of-00001	pytorch_model.bin
model.ckpt-1000000.index		vocab.txt


In [73]:
!ls bert_base/

bert_config.json		     bert_model.ckpt.index  pytorch_model.bin
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta   vocab.txt


In [0]:
import torch

from transformers import BertModel, BertConfig, BertTokenizer

## Prepare the input

In [101]:
text = "CanSino Biologics, headquartered in Tianjin, is close to testing its" \
      "novel coronavirus vaccine in a clinical trial in China. CanSino’s approach " \
      "involves taking a snippet of coronavirus’ genetic code and entwining it " \
      "with a harmless virus, thereby exposing healthy volunteers to the novel " \
      "infection and spurring the production of antibodies. The company said " \
      "this week that Chinese authorities approved its planned trial, which will " \
      "begin as soon as possible. CanSino markets a vaccine for Ebola virus in China." 

bert_base_tokenizer = BertTokenizer.from_pretrained("./biobert/vocab.txt")

# 1. Add the special tokens
marked_text = "[CLS] " + text + " [SEP]"

# 2. Split the sentence into tokens
tokenized_text = bert_base_tokenizer.tokenize(marked_text)

# 3. Map the token strings to their vocabulary indeces
indexed_tokens = bert_base_tokenizer.convert_tokens_to_ids(tokenized_text)

# 4. Display the words with their indices.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

# 5. Segment IDs: 0 for sentence 1 and 1 for sentence 2
segments_ids = [1] * len(tokenized_text)

# 6. convert the lists to tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


[CLS]           101
cans         20,668
##ino         4,559
bio          25,128
##log        13,791
##ics         4,724
,               117
headquartered  9,514
in            1,107
t               189
##ian         1,811
##jin        11,966
,               117
is            1,110
close         1,601
to            1,106
testing       5,193
its           1,157
##nov        15,847
##el          1,883
co            1,884
##rona       15,789
##virus      27,608
vaccine      20,034
in            1,107
a               170
clinical      7,300
trial         3,443
in            1,107
chin          5,144
##a           1,161
.               119
cans         20,668
##ino         4,559
’               787
s               188
approach      3,136
involves      6,808
taking        1,781
a               170
s               188
##ni          2,605
##ppe        20,564
##t           1,204
of            1,104
co            1,884
##rona       15,789
##virus      27,608
’               787
genetic       7,434

## Load weights from default transformers (to compare with the tensorflow model)

In [0]:
model = BertModel.from_pretrained("bert-base-cased")
model.eval()

with torch.no_grad():
  embeddings, cls = model(tokens_tensor, segments_tensors)

transformer_embeddings = embeddings.squeeze()

In [111]:
for word, embedding in zip(tokenized_text, transformer_embeddings):
  print(word, transformer_embeddings[:5])

[CLS] tensor([[ 7.0802e-01, -1.3570e-02, -3.0544e-01,  ..., -3.9950e-01,
          3.9953e-01, -3.8603e-04],
        [ 2.9583e-01, -3.1500e-01,  2.5154e-01,  ..., -4.2627e-01,
          3.0456e-01,  9.8179e-02],
        [ 4.2225e-01, -4.2436e-01,  3.2933e-01,  ..., -4.2615e-01,
         -2.9616e-01,  2.5861e-02],
        [ 6.1493e-01,  5.1981e-02, -1.0528e-01,  ..., -2.2775e-01,
          9.2993e-01,  1.8184e-01],
        [ 9.7992e-01, -4.0849e-01, -8.9345e-01,  ..., -4.0943e-01,
          5.7542e-01, -8.6916e-02]])
cans tensor([[ 7.0802e-01, -1.3570e-02, -3.0544e-01,  ..., -3.9950e-01,
          3.9953e-01, -3.8603e-04],
        [ 2.9583e-01, -3.1500e-01,  2.5154e-01,  ..., -4.2627e-01,
          3.0456e-01,  9.8179e-02],
        [ 4.2225e-01, -4.2436e-01,  3.2933e-01,  ..., -4.2615e-01,
         -2.9616e-01,  2.5861e-02],
        [ 6.1493e-01,  5.1981e-02, -1.0528e-01,  ..., -2.2775e-01,
          9.2993e-01,  1.8184e-01],
        [ 9.7992e-01, -4.0849e-01, -8.9345e-01,  ..., -4.0943

## Load weights from PyTorch converted Tensorflow weights (to compare with Transformers model)

In [0]:
configuration = BertConfig.from_json_file('./bert_base/bert_config.json')
model = BertModel.from_pretrained("./bert_base/pytorch_model.bin", config=configuration)
model.eval()

with torch.no_grad():
  embeddings, cls = model(tokens_tensor, segments_tensors)

tensorflow_embeddings = embeddings.squeeze()

In [113]:
for word, embedding in zip(tokenized_text, tensorflow_embeddings):
  print(word, embedding[:5])

[CLS] tensor([ 0.7080, -0.0136, -0.3054, -0.2832, -0.4734])
cans tensor([ 0.2958, -0.3150,  0.2515, -0.1083, -0.6709])
##ino tensor([ 0.4222, -0.4244,  0.3293,  0.5395,  0.2592])
bio tensor([ 0.6149,  0.0520, -0.1053,  0.5756,  0.1492])
##log tensor([ 0.9799, -0.4085, -0.8934,  0.3584, -0.4468])
##ics tensor([ 0.2792, -0.5877,  0.0782,  0.5845,  0.0549])
, tensor([ 0.3582,  0.2693, -0.1371,  0.7319,  0.1760])
headquartered tensor([ 0.0457, -0.7216,  0.0814,  0.3653, -0.0360])
in tensor([ 0.2864, -0.1087, -0.0292, -0.0799,  0.4864])
t tensor([ 0.1461, -0.1209, -0.0017,  0.1688,  0.0571])
##ian tensor([-0.3160, -0.4995,  0.0915,  0.5655,  0.4278])
##jin tensor([-0.4187, -0.3286,  0.1854,  0.5316,  0.2759])
, tensor([0.8008, 0.5355, 0.1528, 0.4869, 0.7257])
is tensor([0.0882, 0.0598, 0.1472, 0.6301, 0.3900])
close tensor([ 0.0576, -0.0209,  0.1033,  0.2792,  0.0769])
to tensor([ 0.1725, -0.1233, -0.2260,  0.3368,  0.2081])
testing tensor([ 0.3918, -0.1397,  0.1864,  0.1211, -0.3515])
its 

## Load weights BioBERT
Since I have used `transformers-cli convert` to convert the tensorflow model to PyTorch, the rest of the program works just like transformers. Forget Tensorflow even existed.

In [0]:
configuration = BertConfig.from_json_file('./biobert/bert_config.json')
model = BertModel.from_pretrained("./biobert/pytorch_model.bin", config=configuration)
model.eval()

with torch.no_grad():
  embeddings, cls = model(tokens_tensor, segments_tensors)
biobert_embeddings = embeddings.squeeze()

In [115]:
for word, embedding in zip(tokenized_text, biobert_embeddings):
  print(word, embedding[:5])

[CLS] tensor([-0.0065,  0.1404, -0.4794, -0.0418, -0.4687])
cans tensor([ 0.0781, -0.0577, -0.1690,  0.2767,  0.0975])
##ino tensor([ 0.2606, -0.4994, -0.1520,  0.0454,  0.0726])
bio tensor([ 0.3948, -0.0728, -0.2140,  0.1543,  0.1730])
##log tensor([ 0.3731, -0.2170, -0.7370,  0.2493, -0.4223])
##ics tensor([-0.2962,  0.2090, -0.0406,  1.0999,  0.2371])
, tensor([-0.3337, -0.1598, -0.4558,  0.6887,  0.5007])
headquartered tensor([-0.0473, -0.8199, -0.8299,  0.6298, -0.6146])
in tensor([ 0.4575, -0.5467, -0.3971,  0.4630,  0.1766])
t tensor([ 1.7152e-01, -4.4227e-01, -2.3838e-01,  2.4482e-04,  5.0961e-01])
##ian tensor([-0.3334, -0.3899,  0.0256,  0.6071,  0.1383])
##jin tensor([-0.7429, -0.3694,  0.2580,  0.2781,  0.5911])
, tensor([-0.3528,  0.1087, -0.3621,  0.6264,  0.4396])
is tensor([-0.4351, -0.0294, -0.0987,  0.8041, -0.0696])
close tensor([-0.3168,  0.1785, -0.2560,  0.0392, -0.1784])
to tensor([ 0.3835,  0.1897, -0.4827, -0.1329,  0.3971])
testing tensor([ 0.2824, -0.1037,  0

### Proof that the transformer and tensorflow models are exactly the same

In [123]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(transformer_embeddings, tensorflow_embeddings)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.])

In [124]:
### Proof that BioBERT is different
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(transformer_embeddings, biobert_embeddings)

tensor([0.5491, 0.2367, 0.7055, 0.7289, 0.6874, 0.7433, 0.7235, 0.6245, 0.7102,
        0.5944, 0.5964, 0.7147, 0.7552, 0.7649, 0.7119, 0.6661, 0.8036, 0.7680,
        0.6295, 0.6644, 0.6835, 0.6668, 0.7080, 0.7909, 0.7922, 0.7764, 0.7392,
        0.7532, 0.7173, 0.6286, 0.6636, 0.1458, 0.6408, 0.6956, 0.4937, 0.7039,
        0.7931, 0.7793, 0.7573, 0.7381, 0.6466, 0.5461, 0.5848, 0.6744, 0.7274,
        0.7239, 0.6702, 0.7473, 0.6605, 0.7595, 0.7963, 0.7404, 0.6165, 0.5975,
        0.6010, 0.6864, 0.7585, 0.7158, 0.7924, 0.7962, 0.7774, 0.7682, 0.7141,
        0.7472, 0.7796, 0.7659, 0.6960, 0.7697, 0.7322, 0.7723, 0.7588, 0.6802,
        0.6828, 0.7248, 0.7513, 0.7290, 0.7716, 0.1386, 0.7247, 0.7461, 0.6725,
        0.6522, 0.6458, 0.6806, 0.6918, 0.7519, 0.7046, 0.7458, 0.7460, 0.7713,
        0.7560, 0.7307, 0.6832, 0.6655, 0.6582, 0.6282, 0.6579, 0.5215, 0.5910,
        0.7567, 0.6379, 0.7164, 0.7278, 0.7646, 0.7769, 0.7234, 0.6913, 0.5411,
        0.6376, 0.7092, 0.7445, 0.6391, 